# Evaluacion de infraestructura en cadenas logisticas de suministros

Dentro de la cadena de suministro, la logística sirve para la planificación, gestión y control del almacenamiento de bienes, así como los servicios necesarios y el flujo de información generada, que va desde el punto de origen del producto hasta el punto de consumo, y cuyo objetivo es cubrir la demananda de los consumidores.

Dado que un objetivo primordial dentro de la cadena de suministro es trnaportar un producto x a través de un numero n de puntos de infraestructura de forma optima. Es importante evaluar los costos de trasporte entre cada uno de los elementos de la cadena, ya que esto permite identificar puntos donde la inraestructura no es suficiente o la conectividad no es la adecuada. 

Esta aplicación permite dado un numero n de etapas, calcular los costos acumulados de transporte desde el punto de inicio hasta cada uno de los estadíos de la cadena. 


### Suposiciones


- Todas las etapas tienen un identificador único que se llama id
- La red es fija y la tabla se llama red
- La tabla de vértices se llama red_vertices_pgr
- tiene una columna que se llama costo
- tiene campos source y target
- todas las columnas de geometría se llaman geom

Para comenzar a estructurar esta aplicación vamos crear una conexión entre Postgres-postGis y Python con ayuda de las librerías psycopg2 y sqlalchemy. Además vamos a interoperar dataframes y con geodataframes para poder obtener interpolaciones IDW para el diagnostico de cada etapa dentro de la cadena. 

Para continuar es necesario tomar en cuenta que el desarrollo de una aplicación tiene dos etapas importantes:

el __Frontend__ que es la parte que interactúa con los usuarios, por eso decimos que está del lado del _cliente_ y __Backend__ que es la parte que se conecta con la base de datos y el servidor, por eso decimos que el backend corre del lado del _servidor_. Nosotros estaremos trabajando sobre el __Backend__.


Primero hay que crear un enviroment de un archivo .yml

`conda env create -f cadenas.yml` 

In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from functools import reduce
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import from_origin
import io
from subprocess import call

### Parámetros de la conexión
La idea de diseñar este tipo de aplicaciones desde cero, es poder trabajar con ellas tanto en un servidor local como uno remoto. Por lo que, es necesario delimitar los parametros de la conexión con la base de datos como variables. Aquí servidor, usuario, password y base de datos están fijos.

In [2]:
db = 'cadenas'
usr = 'postgres' 
psw = '12345' 
host = 'localhost'

In [3]:
con = psycopg2.connect(database= db, user=usr, password=psw, host=host)
engine = create_engine('postgresql://{}:{}@{}:5432/{}'.format(usr,psw,host,db))

Al estar trabajando sobre el __Backend__ es necesario ir delimitando cuales serán las condiciones que serán manipuladas por el usuario a través del __Frontend__. En este caso, dado que queremos que la aplicación pueda funcionar con cualquier tipo de cadena de suministro con un número de etapas mayor a _n+2_  para comenzar las pruebas nosotros delimitamos una lista, sin embargo al momento de la implementación elusuario deberá cargar la información a la base de datos a través de la interfaz. Lo que la hace adaptativa, en este casi trabajaremos con 4 etapas.  

In [4]:
etapas = ['etapa1', 'etapa2', 'etapa3', 'etapa4','etapa5' ]

### Relaciones entre nodos y etapas

La forma más fácil para trabajar con redes de transporte es a través de pgrouting y postGis, por lo que será necesario estructurar funciones en python que utilicen postGis-postgres y acepten inputs externos de forma iterativa. 

la función node_relations asigna a cada punto de cada etapa el nodo más cercano de la red. Y se crean las tablas con las relaciones:`etapa_i_node`, cada tabla tiene dos columnas, el id de la tabla de etapas y el id de la tabla de nodos.

Como pueden observar si bien estamos trabajando con tablas desde la base de datos, al final lo que devuelve la función es un dataframe de pandas, con el siguiente comando: 
``df = pd.read_sql(sql, engine)`` y lo carga a la base de datos de nuevo: `df.to_sql(table + '_node', engine)` 

In [11]:
def node_relations(table, engine, connection):
    """Toma una tabla de puntos y le agrega una columna con el id del
       nodo más cercano de la red.
       Elimina la tabla de relaciones en caso de que exista.
       Regresa el dataframe con la relación.
    """
    drop_qry = """drop table if exists %(tabla)s_node""" % {"tabla": table}
    curs = connection.cursor()
    curs.execute(drop_qry)
    connection.commit()
    sql = """
            select f.id as id_%(tabla)s, (
              SELECT n.id
              FROM red_vertices_pgr As n
              ORDER BY f.geom <-> n.the_geom LIMIT 1
            )as closest_node
            from %(tabla)s f
          """ % {"tabla": table}
    try:
        df = pd.read_sql(sql, engine)
    except ValueError as e:
        print(e)
    try:
        df.to_sql(table + '_node', engine)
    except ValueError as e:
        print(e)
    return df

Entonces, para agregar todas las tablas de relaciones hacemos:

In [12]:
node_relations_list = []
for etapa in etapas:
    node_relations_list.append(node_relations(etapa, engine, con))

Lo que nos da como resultado una lista con las relaciones entre los untos de cada etapa y el nodo más cercano de la red

In [13]:
node_relations_list[-1].head()

,id_etapa4,closest_node
0,423818,4283
1,482880,11908
2,6254777,24593
3,481138,12911
4,403180,11912


Leemos cada etapa con su identificador de closest_node

In [14]:
etapas_gdfs = []
for etapa in etapas:
    sql = """select a.id, a.geom, b.closest_node
             from %(etapa)s a
             join %(etapa)s_node b
             on a.id = b.id_%(etapa)s""" % {"etapa":etapa}
    etapas_gdfs.append(gpd.GeoDataFrame.from_postgis(sql, con, geom_col='geom'))

### Costos por etapa

Primero la función que calcula el costo acumulado entre etapas sucesivas (esto asume que la lista de etapas está ordenada). Como pueden ver esto también es una función que trabaja con postGis

In [15]:
def stage_cost(source_table, target_table, cost_column):
    params = {'source': source_table, 'target': target_table, 'cost': cost_column }
    qry_str = """SELECT DISTINCT ON (start_vid)
                 start_vid as id_%(source)s, end_vid as id_%(target)s, agg_cost as costo_%(source)s_%(target)s
          FROM   (SELECT * FROM pgr_dijkstraCost(
              'select id, source, target, %(cost)s as cost from red',
              array(select distinct(s.closest_node) from (select e.*, r.closest_node
                                                        from %(source)s e
                                                        join %(source)s_node r
                                                        on e.id = r.id_%(source)s::int) as s),
              array(select distinct(t.closest_node) from (select e.*, r.closest_node
                                                        from %(target)s e
                                                        join %(target)s_node r
                                                        on e.id = r.id_%(target)s::int) as t),
                 directed:=false)
          ) as sub
          ORDER  BY start_vid, agg_cost asc""" % params
    try:
        df = pd.read_sql(qry_str, engine)
    except ValueError as e:
        print(e)
    return df

Posteriormente es necesario calcular las distancias entre los puntos de cada etapa, si lo vemos con ejemplo individual entre dos etamas se calcularía de la siguiente forma:

In [18]:
d0 = stage_cost("etapa1", "etapa2", "costo")
d0.head(10)

,id_etapa1,id_etapa2,costo_etapa1_etapa2
0,4638,8709,71171.567979
1,5077,8473,68720.880610
2,5399,8473,10582.036824
3,6738,24902,25630.037638
4,11122,8709,81400.272837
5,12459,18582,88454.749393
6,16990,21589,930.517860
7,17928,18582,47081.519979
8,19350,21399,16953.527582
9,19352,18582,19401.961503


Y para calcular todas las distancias:

In [19]:
distancias = []
for i, etapa in enumerate(etapas): 
    if i < len(etapas)-1:
        stage = stage_cost(etapa,etapas[i+1], "costo")
        cost_col = list(stage.columns)[-1]
        stage.columns = ['start_' + str(i), 'end_' + str(i), cost_col]
        stage.to_sql('dist_' + etapa, engine, index=False, if_exists='replace')
        distancias.append(stage)

In [20]:
distancias[0].head()

,start_0,end_0,costo_etapa1_etapa2
0,4638,8709,71171.567979
1,5077,8473,68720.880610
2,5399,8473,10582.036824
3,6738,24902,25630.037638
4,11122,8709,81400.272837


In [21]:
distancias[1].head()

,start_1,end_1,costo_etapa2_etapa3
0,3393,4974,84969.108108
1,8033,8031,108.393775
2,8473,8031,14803.982914
3,8709,10352,20540.711121
4,11122,15203,69060.095000


In [22]:
distancias[2].head()

,start_2,end_2,costo_etapa3_etapa4
0,1486,4603,30767.864728
1,4243,1265,42877.538337
2,4974,4603,14130.042401
3,8031,4603,24942.314935
4,8089,4603,78060.136791


Ahora es necesario unirlas en un solo dataframe:

In [23]:
c = pd.merge(distancias[0], distancias[1], left_on='end_0', right_on='start_1')
d = pd.merge(c, distancias[2], left_on='end_1', right_on='start_2')
d.head()

,start_0,end_0,costo_etapa1_etapa2,start_1,end_1,costo_etapa2_etapa3,start_2,end_2,costo_etapa3_etapa4
0,4638,8709,71171.567979,8709,10352,20540.711121,10352,1265,64540.992779
1,11122,8709,81400.272837,8709,10352,20540.711121,10352,1265,64540.992779
2,5077,8473,68720.880610,8473,8031,14803.982914,8031,4603,24942.314935
3,5399,8473,10582.036824,8473,8031,14803.982914,8031,4603,24942.314935
4,6738,24902,25630.037638,24902,1486,173997.857087,1486,4603,30767.864728


Ahora creamos una tabla que contenga solo los puntos de inicio y final ylas distancias entre ellos, además del costo de transporte.

In [24]:
for i, distancia in enumerate(distancias):
    print(i)
    if i == 0:
        costos = pd.merge(distancia, distancias[1], left_on='end_0', right_on='start_1')
        print(list(costos.columns))
        
    elif i < len(distancias) - 1:
        costos = pd.merge(costos, distancias[i+1], left_on='end_' + str(i), right_on='start_' + str(i+1))
        print(list(costos.columns))

0
['start_0', 'end_0', 'costo_etapa1_etapa2', 'start_1', 'end_1', 'costo_etapa2_etapa3']
1
['start_0', 'end_0', 'costo_etapa1_etapa2', 'start_1', 'end_1', 'costo_etapa2_etapa3', 'start_2', 'end_2', 'costo_etapa3_etapa4']
2


In [25]:
costos.head()

,start_0,end_0,costo_etapa1_etapa2,start_1,end_1,costo_etapa2_etapa3,start_2,end_2,costo_etapa3_etapa4
0,4638,8709,71171.567979,8709,10352,20540.711121,10352,1265,64540.992779
1,11122,8709,81400.272837,8709,10352,20540.711121,10352,1265,64540.992779
2,5077,8473,68720.880610,8473,8031,14803.982914,8031,4603,24942.314935
3,5399,8473,10582.036824,8473,8031,14803.982914,8031,4603,24942.314935
4,6738,24902,25630.037638,24902,1486,173997.857087,1486,4603,30767.864728


Para poder hacer la interpolación IDW, necesitamos solo el identificador de los puntos de la primer etapa para hacer un merge con las geometrías y los costos acumulados de las demás. 

In [26]:
costos_acumulados = costos.iloc[:, costos.columns.str.contains('costo_')].cumsum(axis=1)
costos_acumulados = costos_acumulados.merge(costos.iloc[:,[0]], left_index=True, right_index=True)

In [27]:
costos_acumulados.head()

,costo_etapa1_etapa2,costo_etapa2_etapa3,costo_etapa3_etapa4,start_0
0,71171.567979,91712.279101,156253.271880,4638
1,81400.272837,101940.983958,166481.976738,11122
2,68720.880610,83524.863523,108467.178459,5077
3,10582.036824,25386.019738,50328.334673,5399
4,25630.037638,199627.894726,230395.759454,6738


In [28]:
etapas_gdfs[0].head()

,id,geom,closest_node
0,1,POINT (600897.9863290923 1789280.118662399),21715
1,2,POINT (550453.4179119855 1692602.368514441),24137
2,3,POINT (576755.019745179 1740853.210834095),16990
3,4,POINT (556049.3236988379 1880398.360197756),4638
4,5,POINT (488724.6064205672 1849762.308867841),5399


In [29]:
costos_acumulados = etapas_gdfs[0].merge(costos_acumulados, left_on = 'closest_node', right_on = 'start_0')
costos_acumulados.head()

,id,geom,closest_node,costo_etapa1_etapa2,costo_etapa2_etapa3,costo_etapa3_etapa4,start_0
0,1,POINT (600897.9863290923 1789280.118662399),21715,95974.508592,196111.828210,219309.294071,21715
1,2,POINT (550453.4179119855 1692602.368514441),24137,27959.075368,77144.890627,102043.965826,24137
2,3,POINT (576755.019745179 1740853.210834095),16990,930.517860,129933.125472,153130.591333,16990
3,4,POINT (556049.3236988379 1880398.360197756),4638,71171.567979,91712.279101,156253.271880,4638
4,5,POINT (488724.6064205672 1849762.308867841),5399,10582.036824,25386.019738,50328.334673,5399


Ahora hacemos la interpolación y creamos shp's por etapa:

In [32]:
for columna in costos_acumulados:
    if columna.startswith('costo_'):
        write_me = costos_acumulados[['id', 'geom', columna]]
        write_me.columns = ['id', 'geom', 'costo']
        write_me.to_file(driver= 'ESRI Shapefile', filename= "idw/" + columna + '.shp')
        comando = ['gdal_grid', '-zfield', 'costo', '-l', columna, '-a',
           'invdist:power=2.0:smothing=0.0:radius1=0.0:radius2=0.0:angle=0.0:max_points=0:min_points=0:nodata=0.0',
           '-of', 'GTiff', 'idw/'+ columna + '.shp', 'idw/'+ columna+'.tif']
    try:
        call(comando)
    except:
        print('El raster de costos fue creado')

El raster de costos no fue creado
El raster de costos no fue creado
El raster de costos no fue creado


In [29]:
con.close()
#con = psycopg2.connect(database= db, user=usr, password=psw, host=host)